<a href="https://colab.research.google.com/github/Davi-Esmeraldo/TCC/blob/main/TCC_Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests  # Biblioteca para realizar requisições HTTP e acessar páginas da web
from bs4 import BeautifulSoup  # Ferramenta para extrair dados de HTML e XML de forma estruturada
import time  # Utilizado para criar pausas entre as requisições, evitando sobrecarga no servidor
import re  # Biblioteca para trabalhar com expressões regulares e buscar padrões de texto

In [ ]:
# Função para extrair o conteúdo específico de uma portaria
def extrair_conteudo(link):
    try:
        # Faz uma requisição HTTP GET para acessar o conteúdo da página
        response = requests.get(link)

        # Verifica se a resposta foi bem-sucedida (status 200)
        if response.status_code == 200:
            # Interpretar o conteúdo HTML da página
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extrai todo o texto da página, usando '\n' como separador entre linhas e removendo espaços desnecessários
            texto_completo = soup.get_text(separator='\n', strip=True)

            # Define um padrão usando regex para capturar o trecho entre:
            # "Poder Judiciário da União Tribunal de Justiça do Distrito Federal e dos Territórios"
            # até o trecho que antecede "ESTE TEXTO NÃO SUBSTITUI..."
            padrao = (
                r"Poder Judiciário da União\s+Tribunal de Justiça do Distrito Federal e dos Territórios"
                r"(.*?)ESTE TEXTO NÃO SUBSTITUI"
            )

            # Busca o padrão no texto completo da página.
            # A flag re.DOTALL permite que o '.' corresponda a quebras de linha, e re.IGNORECASE ignora maiúsculas/minúsculas.
            match = re.search(padrao, texto_completo, re.DOTALL | re.IGNORECASE)

            if match:
                # Se o padrão for encontrado, retorna o trecho relevante, removendo espaços desnecessários nas bordas
                return match.group(1).strip()
            else:
                # Caso o conteúdo esperado não seja encontrado, exibe uma mensagem de aviso
                print(f"Conteúdo não encontrado no link: {link}")
                return "Conteúdo não encontrado."
        else:
            # Caso a requisição falhe (status diferente de 200), exibe o código de erro
            print(f"Erro ao acessar {link}: {response.status_code}")
            return "Erro ao acessar o conteúdo."

    # Tratamento de exceções para qualquer erro inesperado durante a execução da função
    except Exception as e:
        print(f"Erro ao acessar {link}: {e}")
        return "Erro ao acessar o conteúdo."


In [ ]:
# Função para buscar e extrair links de uma página específica
def extrair_links(url):
    # Faz uma requisição HTTP GET para acessar a página fornecida na URL
    response = requests.get(url)

    # Verifica se a requisição foi bem-sucedida (status 200)
    if response.status_code == 200:
        # Usa BeautifulSoup para interpretar o conteúdo HTML da página
        soup = BeautifulSoup(response.content, 'html.parser')

        # Dicionário para armazenar os dados das portarias, organizados pelo número da portaria
        portarias = {}

        # Percorre todos os elementos <a> (links) da página
        for link in soup.find_all('a'):
            # Obtém o valor do atributo 'href' (link) e o texto associado ao link
            href = link.get('href')
            texto = link.get_text(strip=True)

            # Filtra links que são relevantes, contendo 'portaria' no texto ou PDFs nos links
            if 'portaria' in texto.lower() or 'pdf' in href.lower():
                # Extrai o número da portaria utilizando regex para encontrar dígitos no texto
                numero = re.findall(r'\d+', texto)

                if numero:
                    # Usa o primeiro número encontrado como o número da portaria
                    numero_portaria = numero[0]

                    # Monta o link completo caso o link não esteja absoluto (comece com '/')
                    link_completo = (
                        href if href.startswith('http')
                        else f"https://www.tjdft.jus.br{href}"
                    )

                    # Extrai o conteúdo específico da portaria chamando a função `extrair_conteudo`
                    conteudo = extrair_conteudo(link_completo)

                    # Armazena o texto, link e conteúdo no dicionário, usando o número da portaria como chave
                    portarias[numero_portaria] = {
                        'texto': texto,
                        'link': link_completo,
                        'conteudo': conteudo
                    }

        # Retorna o dicionário contendo todas as portarias encontradas na página
        return portarias
    else:
        # Exibe mensagem de erro caso a requisição não seja bem-sucedida e retorna um dicionário vazio
        print(f"Erro ao acessar {url}: {response.status_code}")
        return {}


In [ ]:
# Função principal para percorrer todas as páginas e extrair informações
def scraping_paginacao(base_url, paginas, delay=1):
    # Dicionário para armazenar todas as portarias coletadas
    todas_portarias = {}

    # Loop para cada página, começando da página 1 até o número total de páginas
    for pagina in range(1, paginas + 1):
        # Monta a URL da página atual, calculando o parâmetro de início com base na página
        url_pag = f"{base_url}?b_start:int={(pagina - 1) * 30}"
        print(f"Acessando: {url_pag}")  # Exibe a URL que está sendo acessada

        # Chama a função 'extrair_links' para coletar as portarias da página atual
        portarias = extrair_links(url_pag)

        # Atualiza o dicionário geral com as portarias extraídas da página atual
        todas_portarias.update(portarias)

        # Pausa de 'delay' segundos entre as requisições para evitar sobrecarga no servidor
        time.sleep(delay)

    # Retorna o dicionário contendo todas as portarias coletadas
    return todas_portarias


In [ ]:
# URL base e número de páginas
base_url = "https://www.tjdft.jus.br/publicacoes/publicacoes-oficiais/portarias-gpr/2024"
numero_paginas = 1  # Total de páginas 52

# Executa o scraping e armazena todas as portarias
todas_portarias = scraping_paginacao(base_url, numero_paginas, delay=0)


Acessando: https://www.tjdft.jus.br/publicacoes/publicacoes-oficiais/portarias-gpr/2024?b_start:int=0


In [ ]:
# Exemplo: Acessando uma portaria específica pelo número

numero_desejado = '1804'
if numero_desejado in todas_portarias:
    print(f"Portaria {numero_desejado}:")
    print(f"Texto: {todas_portarias[numero_desejado]['texto']}")
    print(f"Link: {todas_portarias[numero_desejado]['link']}")
    print(f"Conteúdo:\n{todas_portarias[numero_desejado]['conteudo']}")
else:
    print(f"Portaria {numero_desejado} não encontrada.")


Portaria 1804:
Texto: Portaria GPR 1804 de 17/10/2024
Link: https://www.tjdft.jus.br/publicacoes/publicacoes-oficiais/portarias-gpr/2024/portaria-gpr-1804-de-17-10-2024
Conteúdo:
Gabinete da Presidência
PORTARIA GPR 1804 DE 17 DE OUTUBRO DE 2024
O PRESIDENTE DO TRIBUNAL DE JUSTIÇA DO DISTRITO FEDERAL E DOS TERRITÓRIOS, no uso de sua competência legal e tendo em vista o disposto no processo SEI 0031663/2024,
RESOLVE:
Conceder aposentadoria voluntária integral à servidora Maria Esmeralda de Souza Barros, matrícula 215, ocupante do cargo de Técnico Judiciário, Área Administrativa, Classe C, Padrão 13, do Quadro de Pessoal deste Tribunal de Justiça, com fundamento no art. 3º da
Emenda Constitucional 47/2005
, c/c o art. 3º, caput e § 1º, da
Emenda Constitucional 103/2019
, com as vantagens previstas no art. 67 da
Lei 8.112/1990
, c/c o art. 6º da
Lei 9.624/1998
e com o inciso II do art. 15 da
Medida Provisória 2.225-45/2001
, no art. 3º da
Lei 8.911/1994
, c/c o art. 15 da
Lei 9.527/1997
,